In [17]:
# In a notebook cell, prefix with `!` to run Streamlit in a subprocess:
!streamlit run "/Volumes/TAFOL/Daniel Cooke/Work & Learning/Learning/Python/Streamlit/Quiddler-ScoreSheet/quiddler.py"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.1.178:8501

2025-06-04 21:50:54.478 Uncaught app exception
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 535, in _run_script
    exec(code, module.__dict__)
  File "/Volumes/TAFOL/Daniel Cooke/Work & Learning/Learning/Python/Streamlit/Quiddler-ScoreSheet/quiddler.py", line 76, in <module>
    main()
  File "/Volumes/TAFOL/Daniel Cooke/Work & Learning/Learning/Python/Streamlit/Quiddler-ScoreSheet/quiddler.py", line 54, in main
    scoresheet.render_scoresheet()                # ← call the method
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Volumes/TAFOL/Daniel Cooke/Work & Learning/Learning/Python/Streamlit/Quiddler-ScoreSheet/scoresheet.py", line 240, in render_scoresheet
    self.render_score_editor()
  File "/Volumes/TAFOL/Daniel Cooke